# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Apache Cassandra setup

## You are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font> 

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="event_datafile_new.jpg">

#### Create a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)    

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following questions of the data

##  Query 1

### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

- song_history table contains:
    - artist_name text 
    - song_title text 
    - song_length float 
    - session_id int 
    - item_in_session int
    
- Primary key will be (session_id, item_in_session)
    - session_id is the partition key 
    - item_in_sessionn is the clustering column.    

### Create song_history table

In [8]:
## Query 1:  Give me the artist, song title and song's length 
## in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_history"
query = query + "(artist_name text, \
                  song_title text, \
                  song_length float, \
                  session_id int, \
                  item_in_session int, \
                  PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

### Insert data into song_history table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_history (artist_name, \
                                           song_title, \
                                           song_length, \
                                           session_id, \
                                           item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned 
        ## for each column in the INSERT statement.
        session.execute(query, (line[0], \
                                line[9], \
                                float(line[5]), \
                                int(line[8]), \
                                int(line[3])))

### Verify that the data have been inserted

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, \
                song_title, \
                song_length \
         FROM song_history \
         WHERE session_id = 338 \
         AND item_in_session  = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


##  Query 2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- users table contains:
    - artist_name text 
    - song_title text 
    - item_in_session int 
    - first_name text 
    - last_name text 
    - user_id int 
    - session_id int
    
- Primary key will be (user_id, session_id), item_in_session
    - user_id and session_id make the composite partition key 
    - item_in_sessionn is the clustering column.    

### Create users table

In [11]:
## Query 2: Give me only the following: 
## name of artist, song (sorted by itemInSession)
## and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS users"
query = query + "(artist_name text, \
                  song_title text, \
                  item_in_session int, \
                  first_name text, \
                  last_name text, \
                  user_id int, \
                  session_id int, \
                  PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    
                    

### Insert data into users table

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users (artist_name, \
                                    song_title, \
                                    item_in_session, \
                                    first_name, \
                                    last_name, \
                                    user_id, \
                                    session_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned 
        ## for each column in the INSERT statement.
        session.execute(query, (line[0], \
                                line[9], \
                                int(line[3]), \
                                line[1], \
                                line[4], \
                                int(line[10]), \
                                int(line[8])))

### Verify that the data have been inserted

In [13]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, \
                song_title, \
                first_name,\
                last_name \
         FROM users \
         WHERE user_id = 10 \
         AND session_id  = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


##  Query 3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- song_listened table includes:
    - first_name text 
    - last_name text 
    - song_title text 
    - user_id int
    
- Primary key will be (song_title), user_id
    - song_title is the partition key 
    - user_id is the clustering column.    

### Create song_listened table

In [14]:
## Query 3: Give me every user name (first and last) 
## in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_listened"
query = query + "(first_name text, \
                  last_name text, \
                  song_title text, \
                  user_id int, \
                  PRIMARY KEY ((song_title), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert data into song_listened table

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listened (first_name, \
                                            last_name, \
                                            song_title, \
                                            user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned 
        ## for each column in the INSERT statement.
        session.execute(query, (line[1], \
                                line[4], \
                                line[9],
                                int(line[10])))

### Verify that the data have been inserted

In [16]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT first_name, \
                last_name \
         FROM song_listened \
         WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "DROP TABLE song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_listened"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

### Close the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()